In [46]:
!pip install langchain
!pip install aiohttp

In [13]:
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_community.llms import OpenAI
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity

import os
import asyncio
import time
import numpy as np

In [2]:
os.environ['OPENAI_API_KEY'] = ""
model = ChatOpenAI(temperature=0)

class Temoignage(BaseModel):
    temoignage: str = Field(description="Témoignage original")
    key_words: List[str] = Field(description="Liste des mots clés important du témoignage")

class IdeasList(BaseModel):
    list_of_ideas: List[str] = Field(description="List des idées pour l'utilisateur")


In [63]:
parser = PydanticOutputParser(pydantic_object=Temoignage)

# Keywords extraction
prompt = PromptTemplate(
    template="Donne une liste de mot clé pertinent qui tu peux récupérer du texte.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

temoignage_query = "J'aime faire des poc de nouvelle techno, réaliser des side project. J'aime le python, le backend et la performance"

chain = prompt | model | parser
resp = chain.invoke({"query": temoignage_query})

In [64]:
key_words = resp.key_words
print(key_words)

['poc', 'nouvelle techno', 'side project', 'python', 'backend', 'performance']


In [65]:
parser = PydanticOutputParser(pydantic_object=IdeasList)
all_ideas = []

In [71]:
cursus = "ingénieur"
exp = "10"

async def complet_one(word):
    print("start")
    prompt = PromptTemplate(
        template="Donne une liste d'idée en français de projet pertinent pour un développeur avec un cursus {cursus} et qui a {exp} ans d'expérience dans le monde professionel du développement. Ces idées de projet doivent faire environ 2 à 3 phrases et être en lien avec le mot clé : {word} et doivent également correspondre avec le témoignage de l'utilisateur concernant ce qu'il aime faire : {temoi} \n {format_instructions}\n",
        input_variables=["word", "cursus", "exp", "temoi"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    chain = prompt | model | parser
    resp = await chain.ainvoke({"word": word, "cursus": cursus, "exp": exp, "temoi": temoignage_query})
    print(word, " ok")

    return((word, resp))

In [72]:
async def generate_ideas_concurrently(key_words):
    tasks = [complet_one(word) for word in key_words]
    all_ideas = await asyncio.gather(*tasks)
    return all_ideas

In [73]:
ideas = await generate_ideas_concurrently(key_words)

start
start
start
start
start
start
performance  ok
side project  ok
python  ok
backend  ok
poc  ok
nouvelle techno  ok


In [77]:
print([idea for tab in ideas for idea in tab[1].list_of_ideas])

["Créer un POC utilisant une nouvelle technologie de backend en Python pour améliorer les performances d'une application existante", 'Développer un side project utilisant des technologies de pointe en backend pour explorer de nouvelles fonctionnalités et optimiser les performances', "Réaliser un POC de mise à l'échelle d'une application Python en utilisant des techniques de parallélisme et de distribution pour améliorer les performances", 'Créer une application web en utilisant la nouvelle technologie de conteneurisation Docker pour améliorer les performances du backend en Python.', 'Développer un projet de machine learning en utilisant les dernières bibliothèques Python pour optimiser les performances et la précision des modèles.', "Créer un outil de surveillance de la performance des applications web en utilisant les nouvelles technologies de monitoring et en mettant l'accent sur l'optimisation du backend en Python.", 'Créer une application web performante en utilisant Python comme l

In [76]:
# Préparation des embeddings
all_data = [idea for tab in ideas for idea in tab[1].list_of_ideas]

embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorized_data = embedder.embed_documents(all_data)

assert len(all_data) == len(vectorized_data)
print(vectorized_data)
print("all_data et vectorized_data font bien la même longueur")

#Get five most similar vector

temoignage_query_emb = embedder.embed_query(temoignage_query)

similarities = cosine_similarity([temoignage_query_emb], vectorized_data)[0]

sorted_indices = np.argsort(similarities)[::-1]
bad_sorted_indices = np.argsort(similarities)

top_matches = sorted_indices[:5]
bad_matches = bad_sorted_indices[:5]

for index in top_matches:
    print(f"Good simil : {all_data[index]}, Similarité: {similarities[index]}")

for index in bad_matches:
    print(f"Bad simil: {all_data[index]}, Similarité: {similarities[index]}")

[[-0.07558747380971909, -0.002028173068538308, -0.0873042494058609, -0.04194266349077225, -0.07444629818201065, -0.13885369896888733, -0.01954697072505951, 0.0657966285943985, -0.04090471193194389, -0.07900391519069672, -0.0815030112862587, 0.058576468378305435, 0.08182099461555481, -0.010796702466905117, 0.05432800576090813, -0.02746376395225525, 0.03185467794537544, 0.016953621059656143, 0.08606129884719849, -0.07443687319755554, -0.04974671080708504, 0.022010717540979385, 0.004005531780421734, 0.03108823671936989, -0.0049810647033154964, -0.04196038097143173, -0.014803299680352211, -0.015897585079073906, -0.006947728339582682, -0.030849933624267578, -0.004780692979693413, 0.052057188004255295, -0.0039236880838871, 0.01519845798611641, 0.030825309455394745, 0.05181357264518738, 0.0020195916295051575, -0.09694147109985352, -0.08307325839996338, 0.003605941776186228, -0.03284589573740959, 0.0020693629048764706, -0.0755758285522461, -0.008454634808003902, -0.03567751124501228, -0.041622